In [ ]:
using Plots
using Pkg
using Parameters
using DifferentialEquations

In [ ]:
function calculate_alpha_m(V)
    alpha_m = -1. * (V + 47.0) / (exp(-0.1 * (V+47.)) - 1)
end

function calculate_beta_m(V)
    beta_m = 40. * exp(-0.056 * (V + 72.))
end

function calculate_alpha_h(V)
    alpha_h = 0.126 * exp(-0.25 * (V + 77.))
end

function calculate_beta_h(V)
    beta_h = 1.7/(1 + exp(-0.082 * (V + 22.5)))
end

function calculate_alpha_j(V)
    alpha_j = 0.055 * exp(-0.25 * (V + 78.)) / (1. + exp(-0.2 * (V + 78.)))
end

function calculate_beta_j(V)
    beta_j = 0.3 / (1. + exp(-0.1 * (V + 32.)))
end

function calculate_alpha_d(V)
    alpha_d = 0.095 * exp(-(V - 5.)/100.) / (1. + exp(-(V - 5.) / 13.89))
end

function calculate_beta_d(V)
    beta_d = 0.07 * exp(-(V + 44.) / 59.) / (1. + exp((V + 44.) / 20.))
end

function calculate_alpha_f(V)
    alpha_f = 0.012 * exp(-(V + 28.) / 125.) / (1. + exp((V + 28.) / 6.67))
end

function calculate_beta_f(V)
    beta_f = 0.0065 * exp(-(V + 30.) / 50.) / ( 1. + exp(-(V + 30.) / 5.))
end

function calculate_alpha_x1(V)
    alpha_x1 = 5e-4 * exp((V + 50.) / 12.1) / (1. + exp((V + 50.) / 17.5))
end

function calculate_beta_x1(V)
    beta_x1 = 0.0013 * exp(-(V + 20.) / 16.67) / (1. + exp(-(V + 20.) / 25.) )
end




In [ ]:
function calculate_E_s(Cai)
    E_s = -82.3 - (13.0287 * log(Cai * 0.001))
end

function calculate_i_s(V, d, f, E_s, p)
    @unpack g_s = p
    i_s = g_s * d * f * (V - E_s)
end

function calculate_i_Na(V, m, h, j, p)
    @unpack g_Na, g_Nac, E_Na = p
    i_Na = (g_Na * m^3 * h * j + g_Nac) * (V - E_Na)
end

function calculate_i_x1(V , x1)
   i_x1 = x1 * 8e-3 * (exp(0.004 * (V + 77.)) - 1.) / (exp(0.04 * (V + 35.)))
end

function calculate_i_K1(V)
   i_K_1 = 0.0035 * (4. * (exp(0.04 * (V+85.)) - 1) / (exp(0.08 * (V + 53.)) + exp(0.04 * (V + 53.))) +
        0.2 * (V + 23.) / (1 - exp(-0.04 * (V + 23.)))) 
end

function calculate_Istim(t, p)
    @unpack IstimStart, IstimEnd, IstimPeriod, IstimPulseDuration, IstimAmplitude = p 
    Istim = (IstimEnd>=t>=IstimStart && 
    (t - IstimStart) - floor((t - IstimStart)/IstimPeriod) * IstimPeriod <=IstimPulseDuration) ? IstimAmplitude : 0.0
end

In [ ]:
function compute_algebraic(u, p, t)

    V, m, h, j, Cai, d, f, x1 = u
    
    alpha_m = calculate_alpha_m(V)
    beta_m = calculate_beta_m(V)
    
    alpha_h = calculate_alpha_h(V)
    beta_h = calculate_beta_h(V)
    
    alpha_j = calculate_alpha_j(V)
    beta_j = calculate_beta_j(V)
    
    alpha_d = calculate_alpha_d(V)
    beta_d = calculate_beta_d(V)
    
    alpha_f = calculate_alpha_f(V)
    beta_f = calculate_beta_f(V)
    
    alpha_x1 = calculate_alpha_x1(V)
    beta_x1 = calculate_beta_x1(V)
    
    E_s = calculate_E_s(Cai)
    i_s = calculate_i_s(V, d, f, E_s, p)
    
    i_Na = calculate_i_Na(V, m, h, j, p)
    i_x1= calculate_i_x1(V , x1)
    i_K1 = calculate_i_K1(V)
    Istim =calculate_Istim(t, p)
    
    a = (;
        alpha_m, beta_m, alpha_h, beta_h, alpha_j, beta_j, alpha_d, beta_d,
        alpha_f, beta_f, alpha_x1, beta_x1, E_s, i_s, i_Na, i_x1, i_K1, Istim)

end

In [ ]:
function calculate_d_gate(gate, alpha, beta)
    d_gate = alpha * (1. - gate) - beta * gate
end
function calculate_d_Cai(Cai, i_s)
    d_Cai = -0.01 * i_s / 1. + 0.07 * (0.0001 - Cai)
end
function calculate_d_V(Istim, i_Na, i_s, i_x1, i_K1, p)
    @unpack C = p
    d_V = (Istim - (i_Na+i_s+i_x1+i_K1))/C
end

In [ ]:
function compute_rates!(du, u, p, t)

    V, m, h, j, Cai, d, f, x1 = u
    a = compute_algebraic(u, p, t)
    @unpack alpha_m, beta_m, alpha_h, beta_h, alpha_j, beta_j, alpha_d, beta_d,
    alpha_f, beta_f, alpha_x1, beta_x1, E_s, i_s, i_Na, i_x1, i_K1, Istim = a

    du[1] = calculate_d_V(Istim, i_Na, i_s, i_x1, i_K1, p)  # V  
    du[2] = calculate_d_gate(m, alpha_m, beta_m)            # m
    du[3] = calculate_d_gate(h, alpha_h, beta_h)            # h
    du[4] = calculate_d_gate(j, alpha_j, beta_j)            # j
    du[5] = calculate_d_Cai(Cai, i_s)                       # Cai
    du[6] = calculate_d_gate(d, alpha_d, beta_d)            # d
    du[7] = calculate_d_gate(f, alpha_f, beta_f)            # f
    du[8] = calculate_d_gate(x1, alpha_x1, beta_x1)         # x1
    nothing
end

In [ ]:
s₀ = Dict(
    "V" => -84.624,
    "m" => 0.011,
    "h" => 0.988,
    "j" => 0.975,
    "Cai" => 1e-4,
    "d" => 0.003,
    "f" => 0.994,
    "x1" => 0.0001,
);

In [ ]:
s0 =[-64.624,
    0.011,
    0.988,
    0.975,
    1e-4,
    0.003,
    0.994,
    0.0001,
    ]

In [ ]:
p = Dict(
    "C" => 0.01,
    "g_Na" => 4e-2,
    "E_Na" => 50,
    "g_Nac" => 3e-5,
    "g_s" => 9e-4,
    "IstimStart" => 10,
    "IstimEnd" => 50000,
    "IstimAmplitude" => 0.5,
    "IstimPeriod" => 1000,
    "IstimPulseDuration" => 1)

In [ ]:
tspan = (0., 1500.)
u₀ = deepcopy(s0)

In [ ]:
rhs = ODEFunction(compute_rates!, syms=[:V, :m, :h, :j, :Cai, :d, :f, :x1])
prob = ODEProblem(rhs, u₀, tspan, p)

In [ ]:
sol = solve(prob);

In [ ]:
plot(sol, vars=[:V])